In [2]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import *

print('SAGA code version', SAGA.__version__)

SAGA code version 0.5.0


In [3]:
# import other modules

from collections import defaultdict
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from easyquery import Query

In [4]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)
# NOTE: change the path of `root_dir` to your SAGA dropbox path

saga_database = SAGA.Database(root_dir='/home/yymao/Dropbox/Academia/Collaborations/SAGA')

saga_database_dr12 = SAGA.Database()
saga_database_dr12.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr12_nsa{}.fits.gz'

saga_database_dr14 = SAGA.Database()
saga_database_dr14.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr14_nsa{}.fits.gz'

saga_object_catalog = SAGA.ObjectCatalog(saga_database)
saga_object_catalog_dr12 = SAGA.ObjectCatalog(saga_database_dr12)
saga_object_catalog_dr14 = SAGA.ObjectCatalog(saga_database_dr14)

saga_host_catalog = SAGA.HostCatalog(saga_database)

In [34]:
is_sat = (C.is_sat & C.is_clean)
is_lowz = (C.is_low_z & C.is_clean & C.has_spec)

def get_counts(base_list):
    data = defaultdict(list)

    for t in base_list:
        host_id = t['HOST_NSAID'][0]

        data['SAGA name'].append(saga_host_catalog.id_to_name(host_id))
        data['NSAID'].append(host_id)
        
        data['sat'].append(is_sat.count(t))
        data['low_z'].append(is_lowz.count(t))
        data['total'].append(len(t))

        for i in (-1, 0, 1, 2, 3, 4):
            data['zq_{}'.format(i)].append(Query('ZQUALITY=={}'.format(i)).count(t))

        for i in (-1, 0, 1, 2, 3, 4, 5):
            data['rm_{}'.format(i)].append(Query('REMOVE=={}'.format(i)).count(t))

    return Table(data)

## load base catalogs

In [31]:
# specify columns to load
columns = 'OBJID HOST_NSAID RA DEC SPEC_Z REMOVE SATS ZQUALITY TELNAME SPEC_REPEAT r_mag OBJ_NSAID coord'.split()
#columns.extend(get_sdss_colors())
#columns.extend(map('{}_err'.format, get_sdss_colors()))

cuts = C.is_galaxy & C.fibermag_r_cut

# load base catalogs (will take a while)
catalogs_orig = saga_object_catalog.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')
catalogs_dr12 = saga_object_catalog_dr12.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')
catalogs_dr14 = saga_object_catalog_dr14.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')

## tables of counts

In [35]:
# Current base catalogs
get_counts(catalogs_orig)

SAGA name,NSAID,sat,low_z,total,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,37300,33770,0,109,37,474,2910,34426,0,11,154,1687,116,906
Odyssey,147100,9,14,36854,33868,0,117,53,307,2509,33723,0,4,153,2105,145,724
Dune,165536,1,8,37270,32808,0,171,128,388,3775,34734,0,17,155,1389,141,834
AnaK,61945,3,7,38481,35070,0,166,41,573,2631,35281,0,18,235,1963,139,845
Narnia,132339,2,5,39272,36935,0,146,20,212,1959,35998,0,6,203,2150,104,811
OBrother,149781,4,4,34745,33147,0,91,41,162,1304,31563,0,6,98,2062,47,969
StarTrek,33446,2,5,35451,33578,0,102,8,124,1639,32015,0,9,55,2478,102,792
Catch22,150887,6,11,33640,31694,0,254,144,156,1392,30641,0,19,80,1596,84,1220
ScoobyDoo,161174,4,8,35655,34573,0,37,2,43,1000,33441,0,2,71,1316,63,762


In [36]:
# new base catalogs with DR12
get_counts(catalogs_dr12)

SAGA name,NSAID,sat,low_z,total,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,37300,33772,0,110,36,474,2908,34436,0,11,155,1680,112,906
Odyssey,147100,9,15,36855,33853,0,118,54,310,2520,33733,0,4,158,2088,148,724
Dune,165536,1,8,37270,32804,0,175,129,387,3775,34741,0,16,155,1381,143,834
AnaK,61945,3,7,38481,35062,0,169,41,577,2632,35272,0,21,235,1971,137,845
Narnia,132339,2,5,39272,36933,0,146,20,214,1959,35998,0,6,209,2149,99,811
OBrother,149781,4,5,34745,33152,0,90,41,162,1300,31557,0,6,102,2064,47,969
StarTrek,33446,2,5,35451,33579,0,102,8,123,1639,32017,0,11,56,2477,98,792
Catch22,150887,6,11,33640,31696,0,258,144,156,1386,30651,0,24,80,1594,71,1220
ScoobyDoo,161174,4,8,35655,34576,0,36,2,43,998,33439,0,2,71,1320,61,762


In [37]:
# new base catalogs with DR14
get_counts(catalogs_dr14)

SAGA name,NSAID,sat,low_z,total,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,37202,33710,0,108,37,476,2871,33960,0,11,154,1664,511,902
Odyssey,147100,9,15,36798,33839,0,120,54,307,2478,33236,0,4,154,2073,608,723
Dune,165536,1,8,37147,32696,0,170,129,388,3764,34282,0,16,155,1365,500,829
AnaK,61945,3,7,38292,34632,0,267,41,577,2775,34007,0,18,234,1937,1257,839
Narnia,132339,2,5,39223,37038,0,162,21,214,1788,35210,0,6,203,2145,850,809
OBrother,149781,4,5,34696,33124,0,87,41,162,1282,31277,0,6,98,2049,303,963
StarTrek,33446,2,5,35431,33574,0,105,8,123,1621,31400,0,9,56,2472,703,791
Catch22,150887,6,11,33638,31715,0,248,144,156,1375,30188,0,21,80,1592,536,1221
ScoobyDoo,161174,4,8,35885,34828,0,33,2,43,979,33190,0,2,71,1335,517,770


## check satellites

In [39]:
vstack([is_sat.filter(t)[t.colnames[:-1]] for t in catalogs_dr12]).show_in_notebook()

idx,OBJID,HOST_NSAID,RA,DEC,SPEC_Z,REMOVE,SATS,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,OBJ_NSAID
0,1237665566078402826,166313,233.787037531,16.8704375988,0.006671,-1,1,4,MMT,MMT+NSA+AAT+SDSS,15.4017,124222
1,1237665565541728490,166313,234.132926177,16.4404552908,0.0063225,-1,1,4,NSA,NSA+SDSS,14.2841,166312
2,1237662224092364842,147100,247.877387573,20.0936248407,0.008288,-1,1,4,MMT,MMT+AAT+SDSS,16.9963,-1
3,1237662224092496776,147100,248.195368592,19.8670126788,0.00824,-1,1,3,MMT,MMT,18.5589,-1
4,1237662662147310256,147100,247.825892241,20.2108787761,0.00861264,-1,1,4,NSA,SDSS+NSA+WIYN+DEIMOS+AAT,16.9068,147082
5,1237662662147571761,147100,248.393225697,19.9461400824,0.008172,-1,1,4,MMT,MMT+NSA+AAT+SDSS,15.6295,68553
6,1237662698115433445,147100,248.16342684,19.792207594,0.007471,-1,1,4,MMT,MMT,20.3247,-1
7,1237662224092299399,147100,247.84002995,20.1840758135,0.00855347,-1,1,4,NSA,NSA+SDSS,13.2991,68560
8,1237662662147638034,147100,248.580638518,19.7208013394,0.00818,-1,1,4,AAT,MMT+AAT+WIYN,18.2255,-1
9,1237662698115432544,147100,248.051339587,19.6957396993,0.008224,-1,1,4,MMT,MMT+AAT,16.6417,-1


## difference in low-z galaxy count is due to whether or not host is counted

In [38]:
for to, tn in zip(catalogs_orig, catalogs_dr12):
    # make sure we are comparing the same host
    assert to['HOST_NSAID'][0] == tn['HOST_NSAID'][0]
    
    # check if satellite number is the same
    if is_sat.count(to) != is_sat.count(tn):
        print('different satellite count for {}'.format(tn['HOST_NSAID'][0]))
    
    if is_lowz.count(to) != is_lowz.count(tn):
        print('different low-z galaxy count for {}: {} (orig) vs. {} (new)'.format(\
                    tn['HOST_NSAID'][0], is_lowz.count(to), is_lowz.count(tn)))
        tn_lowz = is_lowz.filter(tn)
        to_lowz = is_lowz.filter(to)
        for gal in tn_lowz:
            if len(find_near_coord(to_lowz, gal['coord'], 1.0)) == 0:
                print(' missing obj NSAID', gal['OBJ_NSAID'])

different low-z galaxy count for 147100: 14 (orig) vs. 15 (new)
 missing obj NSAID 147100
different low-z galaxy count for 149781: 4 (orig) vs. 5 (new)
 missing obj NSAID 149781
different low-z galaxy count for 145729: 4 (orig) vs. 5 (new)
 missing obj NSAID 145729


## checking some remove-4 flag objects

In [8]:
host_nsa = 166313
columns = 'OBJID HOST_NSAID RA DEC SPEC_Z REMOVE SATS ZQUALITY TELNAME SPEC_REPEAT PETRORAD_R PETRORADERR_R r_mag OBJ_NSAID coord'.split()
bo = saga_object_catalog.load(host_nsa, columns=columns).pop()
bn = saga_object_catalog_dr12.load(host_nsa, columns=columns).pop()

In [9]:
bn_objid_rm4 = Query('REMOVE==4').filter(bn)['OBJID']
bo_objid_rm4 = Query('REMOVE==4').filter(bo)['OBJID']

kn = bn_objid_rm4[np.in1d(bn_objid_rm4, bo_objid_rm4, invert=True)]
ko = bo_objid_rm4[np.in1d(bo_objid_rm4, bn_objid_rm4, invert=True)]

print(len(bn_objid_rm4), len(bo_objid_rm4))
print(len(kn), len(ko))

269 275
2 8


In [40]:
i = kn[1]
display(find_near_objid(bo, i, 15))
display(find_near_objid(bn, i, 15))

OBJID,HOST_NSAID,RA,DEC,SPEC_Z,REMOVE,SATS,ZQUALITY,TELNAME,SPEC_REPEAT,PETRORAD_R,PETRORADERR_R,r_mag,OBJ_NSAID,coord
,,,,,,,,,,,,,,"deg,deg"
int64,int64,float64,float64,float32,int64,int64,int64,str6,str48,float32,float32,float32,int64,object
1237668367995765229,166313,234.170721217,16.2523974097,-1.0,-1,-1,-1,,,1.36431,0.233502,21.5549,-1,"234.17072121654434,16.252397409748877"
1237668367995765227,166313,234.17075988,16.2527533213,-1.0,-1,-1,-1,,,4.13027,1.39904,19.5524,-1,"234.17075987995355,16.25275332126862"
1237668367995765228,166313,234.169885047,16.2515194624,0.195815,-1,0,4,MMT,AAT+MMT,1.411,0.097014,20.3851,-1,"234.16988504667972,16.251519462432398"
1237668367995765577,166313,234.173247592,16.2532916526,-1.0,-1,-1,-1,,,1.88713,0.251973,21.8033,-1,"234.17324759204493,16.253291652637596"


OBJID,HOST_NSAID,RA,DEC,SPEC_Z,REMOVE,SATS,ZQUALITY,TELNAME,SPEC_REPEAT,PETRORAD_R,PETRORADERR_R,r_mag,OBJ_NSAID,coord
,,,,,,,,,,,,,,"deg,deg"
int64,int32,float64,float64,float32,int16,int16,int16,str6,str48,float32,float32,float32,int32,object
1237668367995765229,166313,234.170721217,16.2523974097,-1.0,4,-1,-1,,,1.36431,0.233502,21.5549,-1,"234.17072121654434,16.252397409748877"
1237668367995765227,166313,234.17075988,16.2527533213,0.196171,-1,0,4,MMT,MMT+AAT,4.13027,1.39904,19.5524,-1,"234.17075987995355,16.25275332126862"
1237668367995765228,166313,234.169885047,16.2515194624,0.195815,-1,0,4,MMT,MMT,1.411,0.097014,20.3851,-1,"234.16988504667972,16.251519462432398"
1237668367995765577,166313,234.173247592,16.2532916526,-1.0,-1,-1,-1,,,1.88713,0.251973,21.8033,-1,"234.17324759204493,16.253291652637596"
